![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

In [0]:
from johnsnowlabs import nlp, legal, viz

🚨 Your Spark-OCR is outdated, installed==4.3.0 but latest version==4.3.1
You can run nlp.install() to update Spark-OCR

#🔎 Legal Relation Extraction(RE) and Zero-shot Relation Extraction

📚Legal relation extraction is a task in natural language processing (NLP) that involves extracting relationships between entities in legal documents. These relationships can be between people, organizations, or legal concepts.

📚Legal relation extraction is useful for a variety of purposes, including legal research, contract analysis, and legal case management. For example, legal relation extraction can be used to identify relationships between parties in a contract, such as the buyer and seller, or to extract clauses in a contract that outline certain obligations or rights.

##✔️ Pretrained Relation Extraction Models and Pipelines for Legal

Here are the list of pretrained Relation Extraction models and pipelines:

📚**Relation Extraction Models**

|index|model|
|-----:|:-----|
| 1| [Legal Relation Extraction (Parties, Alias, Dates, Document Type) (Small, Bidirectional)](https://nlp.johnsnowlabs.com/2022/08/12/legre_contract_doc_parties_en_3_2.html)  | 
| 2| [Legal Relation Extraction (Parties, Alias, Dates, Document Type) (Medium, Undirectional)](https://nlp.johnsnowlabs.com/2022/11/02/legre_contract_doc_parties_md_en.html)  | 
| 3| [Legal Relation Extraction (Alias)](https://nlp.johnsnowlabs.com/2022/08/17/legre_org_prod_alias_en_3_2.html)  |
| 4| [Legal Relation Extraction (Whereas) (Small, Bidirectional)](https://nlp.johnsnowlabs.com/2022/08/24/legre_whereas_en.html)  | 
| 5| [Legal Relation Extraction (Whereas) (Medium, Unidirectional)](https://nlp.johnsnowlabs.com/2022/11/09/legre_whereas_md_en.html)  | 
| 6| [Legal Relation Extraction (Indemnification) (Small, Bidirectional)](https://nlp.johnsnowlabs.com/2022/09/28/legre_indemnifications_en.html)  |
| 7| [Legal Relation Extraction (Indemnification) (Medium, Unidirectional)](https://nlp.johnsnowlabs.com/2022/11/09/legre_indemnifications_md_en.html)  | 
| 8| [Legal Relation Extraction (Confidentiality) (Small, Bidirectional)](https://nlp.johnsnowlabs.com/2022/10/18/legre_confidentiality_en.html)  |
| 9| [Legal Relation Extraction (Confidentiality) (Medium, Unidirectional)](https://nlp.johnsnowlabs.com/2022/11/09/legre_confidentiality_md_en.html)  |
| 10| [Legal Relation Extraction (Warranty)](https://nlp.johnsnowlabs.com/2022/10/19/legre_warranty_en.html)  |
| 11| [Legal Relation Extraction (Grants) (Medium, Unidirectional)](https://nlp.johnsnowlabs.com/2022/11/09/legre_grants_md_en.html)  |
| 12| [(Obligations) (Medium, Unidirectional)](https://nlp.johnsnowlabs.com/2022/11/03/legre_obligations_md_en.html)  |
| 13| [Legal Relation Extraction (Notice Clause)](https://nlp.johnsnowlabs.com/2022/12/17/legre_notice_clause_xs_en.html)  |
| 14| [Legal Zero-shot Relation Extraction](https://nlp.johnsnowlabs.com/2022/08/22/legre_zero_shot_en_3_2.html)  |
| 15| [Pretrained Pipeline(Whereas)](https://nlp.johnsnowlabs.com/2022/08/24/legpipe_whereas_en.html)  |

##🔎 NER and Relation Extraction
NER only extracts isolated entities by itself. But you can combine some NER with specific Relation Extraction Annotators trained for them, to retrieve if the entities are related to each other.

Let's suppose we want to extract information about **PARTIES**, **ALIAS**, **DATES** and **DOCUMENT_TYPES**. If we don't know where that information is in the document, we can use Text Classifiers to find it.

Firstly, we will download sample dataset and do all progress on it.

In [0]:
! wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/legal-nlp/data/intellectual_property_agreement.txt

dbutils.fs.cp("file:/databricks/driver/intellectual_property_agreement.txt", "dbfs:/") 

Out[2]: True

In [0]:
with open('/dbfs/intellectual_property_agreement.txt', 'r') as f:
  agreement = f.read()
print(agreement[:1500])

Exhibit 10.2

Execution Version

INTELLECTUAL PROPERTY AGREEMENT

This INTELLECTUAL PROPERTY AGREEMENT (this "Agreement"), dated as of December 31, 2018 (the "Effective Date") is entered into by and between Armstrong Flooring, Inc., a Delaware corporation ("Seller") and AFI Licensing LLC, a Delaware limited liability company ("Licensing" and together with Seller, "Arizona") and AHF Holding, Inc. (formerly known as Tarzan HoldCo, Inc.), a Delaware corporation ("Buyer") and Armstrong Hardwood Flooring Company, a Tennessee corporation (the "Company" and together with Buyer the "Buyer Entities") (each of Arizona on the one hand and the Buyer Entities on the other hand, a "Party" and collectively, the "Parties").

WHEREAS, Seller and Buyer have entered into that certain Stock Purchase Agreement, dated November 14, 2018 (the "Stock Purchase Agreement"); WHEREAS, pursuant to the Stock Purchase Agreement, Seller has agreed to sell and transfer, and Buyer has agreed to purchase and acquire, all of Seller's right, title and interest in and to Armstrong Wood Products, Inc., a Delaware corporation ("AWP") and its Subsidiaries, the Company and HomerWood Hardwood Flooring Company, a Delaware corporation ("HHFC," and together with the Company, the "Company Subsidiaries" and together with AWP, the "Company Entities" and each a "Company Entity") by way of a purchase by Buyer and sale by Seller of the Shares, all upon the terms and condition set forth therein;

WHEREAS, Arizona owns certain Co

📜We have lots of classification models to get relevant pages or clauses. You can find any of them in our [Models Hub](https://nlp.johnsnowlabs.com/models]). 

Why do we use this?Because, we don't need to run all document with any pretraiend models.

Firstly,  we will split all document to page or paragraphs.

Here, we get the paraghraps from the entire agreement. As you see above, paraghraps are splitted with `\n\n` in the agreement. So we use `setCustomBounds` parameter in `TextSplitter`.

In [0]:
document_assembler = nlp.DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

text_splitter = legal.TextSplitter() \
    .setInputCols(["document"]) \
    .setOutputCol("paragraphs")\
    .setCustomBounds(["\n\n"])\
    .setUseCustomBoundsOnly(True)\
    .setExplodeSentences(True)

nlp_pipeline = nlp.Pipeline(stages=[
    document_assembler,
    text_splitter])

empty_df = spark.createDataFrame([[""]]).toDF("text")

model = nlp_pipeline.fit(empty_df)

light_model = nlp.LightPipeline(model)


In [0]:
result = light_model.annotate(agreement)

paragraphs = result['paragraphs']

paragraphs[:10]

Out[37]: ['Exhibit 10.2',
 'Execution Version',
 'INTELLECTUAL PROPERTY AGREEMENT',
 'This INTELLECTUAL PROPERTY AGREEMENT (this "Agreement"), dated as of December 31, 2018 (the "Effective Date") is entered into by and between Armstrong Flooring, Inc., a Delaware corporation ("Seller") and AFI Licensing LLC, a Delaware limited liability company ("Licensing" and together with Seller, "Arizona") and AHF Holding, Inc. (formerly known as Tarzan HoldCo, Inc.), a Delaware corporation ("Buyer") and Armstrong Hardwood Flooring Company, a Tennessee corporation (the "Company" and together with Buyer the "Buyer Entities") (each of Arizona on the one hand and the Buyer Entities on the other hand, a "Party" and collectively, the "Parties").',
 'WHEREAS, Seller and Buyer have entered into that certain Stock Purchase Agreement, dated November 14, 2018 (the "Stock Purchase Agreement"); WHEREAS, pursuant to the Stock Purchase Agreement, Seller has agreed to sell and transfer, and Buyer has agreed to purchase and acquire, all of Seller\'s right, title and interest in and to Armstrong Wood Products, Inc., a Delaware corporation ("AWP") and its Subsidiaries, the Company and HomerWood Hardwood Flooring Company, a Delaware corporation ("HHFC," and together with the Company, the "Company Subsidiaries" and together with AWP, the "Company Entities" and each a "Company Entity") by way of a purchase by Buyer and sale by Seller of the Shares, all upon the terms and condition set forth therein;',
 "WHEREAS, Arizona owns certain Copyrights, Know-How, Patents and Trademarks which may be used in the Company Field, and in connection with the transactions contemplated by the Stock Purchase Agreement the Company desires to acquire all of Arizona's right, title and interest in and to such Intellectual Property used exclusively in the Company Field, and obtain a license from Arizona to use other such Intellectual Property on the terms and subject to the conditions set forth herein;",
 'WHEREAS, Seller is signatory to the Trademark License Agreement pursuant to which Seller obtains a license to the Arizona Licensed Trademarks;',
 'WHEREAS, the Company desires to obtain a sublicense to use the Arizona Licensed Trademarks in the Company Field;',
 'WHEREAS, Arizona has obtained consent from all counterparties to the Trademark License Agreement to grant to the Company the sublicenses to the Arizona Licensed Trademarks included in this Agreement; and',
 'WHEREAS, the Company Entities own certain Copyrights and Know-How which may be used in the Arizona Field, and in connection with the transactions contemplated by the Stock Purchase Agreement, Arizona desires to obtain a license from the Company Entities to use such Intellectual Property on the terms and subject to the conditions set forth herein.']

In [0]:
len(paragraphs)

Out[38]: 171

In [0]:
candidates = [paragraphs[3]]

candidates

Out[39]: ['This INTELLECTUAL PROPERTY AGREEMENT (this "Agreement"), dated as of December 31, 2018 (the "Effective Date") is entered into by and between Armstrong Flooring, Inc., a Delaware corporation ("Seller") and AFI Licensing LLC, a Delaware limited liability company ("Licensing" and together with Seller, "Arizona") and AHF Holding, Inc. (formerly known as Tarzan HoldCo, Inc.), a Delaware corporation ("Buyer") and Armstrong Hardwood Flooring Company, a Tennessee corporation (the "Company" and together with Buyer the "Buyer Entities") (each of Arizona on the one hand and the Buyer Entities on the other hand, a "Party" and collectively, the "Parties").']

##✔️ Using Text Classification to Find Relevant Parts of the Document

In this case, we know paragraphs 4 is the paragraph with introduction of the agreement. However, let's suppose we don't know it. So, we can use Clasue Classification.

To check introduction of the agreement , we have a specific model called `legclf_introduction_clause_cuad`

In [0]:
# Text Classifier

def generic_clf_pipeline(model_name):

  """This pipeline allows you to use different classification models to understand if an input text is of a specific class or is something else.
  It will be used to check where the introduction of agreement is, where the WHEREAS clause are, and etc."""

  document_assembler = nlp.DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

  embeddings = nlp.BertSentenceEmbeddings.pretrained("sent_bert_base_cased", "en")\
    .setInputCols("document") \
    .setOutputCol("sentence_embeddings")

  doc_classifier = legal.ClassifierDLModel.pretrained(model_name, "en", "legal/models")\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("category")

  pipeline = nlp.Pipeline(stages=[
    document_assembler, 
    embeddings,
    doc_classifier
  ])

  empty_df = spark.createDataFrame([[""]]).toDF("text")

  model = pipeline.fit(empty_df)

  return model


In [0]:
model_name = "legclf_introduction_clause_cuad"

model = generic_clf_pipeline(model_name)

df = spark.createDataFrame([candidates]).toDF("text")

result = model.transform(df)

sent_bert_base_cased download started this may take some time.
Approximate size to download 389.1 MB
[ | ][OK!]
legclf_introduction_clause_cuad download started this may take some time.
[ | ][OK!]

In [0]:
result.select('category.result').show()

+--------------+
 result|
+--------------+
[introduction]|
+--------------+

Confirmed, paragraphs 4 is introduction of the agreement!

##📌 Extract Relations Between Parties in an Agreement

Main component to carry out information extraction and extract entities from texts. 

This time we will use the `legner_contract_doc_parties` model, which is trained to extract many entities from contracts.

After that, we will extract the relations between these entities using `legre_contract_doc_parties` model.

In [0]:
# Relation Extraction Pipeline Function

def generic_re_pipeline(ner_model, re_model):

  """This pipeline allows you to get relations between the entities."""

  document_assembler = nlp.DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

  text_splitter = legal.TextSplitter()\
      .setInputCols(["document"])\
      .setOutputCol("sentence")

  tokenizer = nlp.Tokenizer()\
      .setInputCols(["sentence"])\
      .setOutputCol("token")

  embeddings = nlp.RoBertaEmbeddings.pretrained("roberta_embeddings_legal_roberta_base", "en") \
      .setInputCols("sentence", "token") \
      .setOutputCol("embeddings")\
      .setMaxSentenceLength(512)

  ner_model = legal.NerModel.pretrained(ner_model, "en", "legal/models")\
      .setInputCols(["sentence", "token", "embeddings"])\
      .setOutputCol("ner")

  ner_converter = nlp.NerConverter()\
      .setInputCols(["sentence","token","ner"])\
      .setOutputCol("ner_chunk")

  """
  ONLY NEEDED IF YOU WANT TO FILTER RELATION PAIRS OR SYNTACTIC DISTANCE
  pos_tagger = nlp.PerceptronModel().pretrained() \
      .setInputCols(["document", "token"])\
      .setOutputCol("pos_tags")

  dependency_parser = nlp.DependencyParserModel() \
      .pretrained("dependency_conllu", "en") \
      .setInputCols(["document", "pos_tags", "token"]) \
      .setOutputCol("dependencies")

  Set a filter on pairs of named entities which will be treated as relation candidates
  re_filter = legal.RENerChunksFilter()\
      .setInputCols(["ner_chunk", "dependencies"])\
      .setOutputCol("re_ner_chunks")\
      .setMaxSyntacticDistance(7)\
      .setRelationPairs(['PARTY-ALIAS', 'DOC-PARTY', 'DOC-EFFDATE'])
  """
  re_model = legal.RelationExtractionDLModel.pretrained(re_model, "en", "legal/models")\
      .setPredictionThreshold(0.1)\
      .setInputCols(["ner_chunk", "sentence"])\
      .setOutputCol("relations")

  pipeline = nlp.Pipeline(stages=[
          document_assembler,
          text_splitter,
          tokenizer,
          embeddings,
          ner_model,
          ner_converter,
          re_model
          ])
  empty_df = spark.createDataFrame([[""]]).toDF("text")

  model = pipeline.fit(empty_df)

  return model

In [0]:
# Create Generic Function to Show Relations in Dataframe

import pandas as pd
def get_relations_df (results, col='relations'):
    rel_pairs=[]
    for i in range(len(results)):
        for rel in results[i][col]:
            rel_pairs.append((
              rel.result, 
              rel.metadata['entity1'], 
              rel.metadata['entity1_begin'],
              rel.metadata['entity1_end'],
              rel.metadata['chunk1'], 
              rel.metadata['entity2'],
              rel.metadata['entity2_begin'],
              rel.metadata['entity2_end'],
              rel.metadata['chunk2'], 
              rel.metadata['confidence']
          ))
    rel_df = pd.DataFrame(rel_pairs, columns=['relation','entity1','entity1_begin','entity1_end','chunk1','entity2','entity2_begin','entity2_end','chunk2', 'confidence'])
    return rel_df

📜As an output, you will get the relations linking the different concepts together, if such relation exists. The list of relations is:

- **dated_as**: A document has an effective date
- **has_alias**: The alias of a party all along the document
- **has_collective_alias**: An alias hold by several parties at the same time
- **signed_by**: Between a party and the document they signed

In [0]:
ner_model = "legner_contract_doc_parties"

re_model = "legre_contract_doc_parties"

model = generic_re_pipeline(ner_model, re_model)

light_model = nlp.LightPipeline(model)

result = light_model.fullAnnotate(candidates)


roberta_embeddings_legal_roberta_base download started this may take some time.
Approximate size to download 447.2 MB
[ | ][OK!]
legner_contract_doc_parties download started this may take some time.
[ | ][OK!]
legre_contract_doc_parties download started this may take some time.
[ | ][OK!]

In [0]:
# Recognized entities
pd.DataFrame([(x.result, x.metadata["entity"]) for x in result[0]["ner_chunk"]], columns=["text", "ner"])

Out[46]:

,text,ner
0,INTELLECTUAL PROPERTY AGREEMENT,DOC
1,"December 31, 2018",EFFDATE
2,"Armstrong Flooring, Inc",PARTY
3,Seller,ALIAS
4,AFI Licensing LLC,PARTY
5,Licensing,ALIAS
6,Seller,ALIAS
7,Arizona,ALIAS
8,"AHF Holding, Inc",PARTY
9,Buyer,ALIAS


In [0]:
rel_df = get_relations_df(result)

rel_df[rel_df["relation"] != "no_rel"]

Out[47]:

,relation,entity1,entity1_begin,entity1_end,chunk1,entity2,entity2_begin,entity2_end,chunk2,confidence
0,dated_as,DOC,5,35,INTELLECTUAL PROPERTY AGREEMENT,EFFDATE,69,85,"December 31, 2018",0.9856822
1,signed_by,DOC,5,35,INTELLECTUAL PROPERTY AGREEMENT,PARTY,141,163,"Armstrong Flooring, Inc",0.78165114
3,signed_by,DOC,5,35,INTELLECTUAL PROPERTY AGREEMENT,PARTY,205,221,AFI Licensing LLC,0.5352147
15,has_alias,PARTY,141,163,"Armstrong Flooring, Inc",ALIAS,192,197,Seller,0.89620024
26,has_alias,PARTY,205,221,AFI Licensing LLC,ALIAS,263,271,Licensing,0.9518907
33,has_collective_alias,ALIAS,292,297,Seller,ALIAS,301,307,Arizona,0.8934925
42,has_alias,PARTY,411,445,Armstrong Hardwood Flooring Company,ALIAS,478,484,Company,0.98353046
51,has_collective_alias,ALIAS,505,509,Buyer,ALIAS,516,529,Buyer Entities,0.72171456
56,has_collective_alias,ALIAS,611,615,Party,ALIAS,641,647,Parties,0.5040901


###✔️ Visualization of Extracted Relations

In [0]:
# from sparknlp_display import RelationExtractionVisualizer

re_vis = viz.RelationExtractionVisualizer()

vis = re_vis.display(result = result[0],
           relation_col = "relations",
           document_col = "document",
           exclude_relations = ["no_rel"],
           show_relations=True,
                     return_html=True
           )

displayHTML(vis)

This INTELLECTUAL PROPERTY AGREEMENT DOC (this "Agreement"), dated as of December 31, 2018 EFFDATE (the "Effective Date") is entered into by and between Armstrong Flooring, Inc PARTY ., a Delaware corporation (" Seller ALIAS ") and AFI Licensing LLC PARTY , a Delaware limited liability company (" Licensing ALIAS " and together with Seller ALIAS , " Arizona ALIAS ") and AHF Holding, Inc. (formerly known as Tarzan HoldCo, Inc.), a Delaware corporation ("Buyer") and Armstrong Hardwood Flooring Company PARTY , a Tennessee corporation (the " Company ALIAS " and together with Buyer ALIAS the " Buyer Entities ALIAS ") (each of Arizona on the one hand and the Buyer Entities on the other hand, a " Party ALIAS " and collectively, the " Parties ALIAS "). <polyline fill="none" points="498.0,355.0 496.71428571428567,353.7960798646822 495.42857142857133,352.59312871337613 494.1428571428572,351.3921155300938 492.85714285714283,350.19400929884654 491.5714285714286,348.99977900364644 490.2857142857142,347.81039362850504 489.0000000000001,346.6268221574345 487.7142857142858,345.4500335744461 486.4285714285715,344.2809968635519 485.1428571428571,343.1206810087633 483.8571428571428,341.97005499409255 482.5714285714285,340.8300878035512 481.2857142857142,339.701748421151 480.0,338.5860058309038 478.7142857142858,337.4838290168213 477.42857142857144,336.3961869629151 476.1428571428573,335.3240486531973 474.85714285714283,334.2683830716793 473.5714285714286,333.23015920237316 472.28571428571416,332.21034602929046 471.0,331.20991253644314 469.7142857142857,330.22982770784284 468.42857142857144,329.2710605275013 467.14285714285717,328.3345799794304 465.8571428571428,327.4213550476417 464.57142857142856,326.5323547161472 463.2857142857142,325.6685479689586 462.0000000000001,324.8309037900875 460.7142857142857,324.0203911635458 459.42857142857144,323.2379790733453 458.1428571428572,322.4846365034977 456.85714285714283,321.7613324380148 455.57142857142856,321.06903586090834 454.2857142857143,320.40871575619 453.0,319.78134110787175 451.71428571428567,319.1878808999652 450.42857142857144,318.6293041164821 449.1428571428571,318.1065797414343 447.8571428571429,317.6206767588335 446.57142857142856,317.1725641526915 445.28571428571433,316.76321090702004 444.0,316.3935860058309 442.71428571428567,316.06465843313583 441.42857142857144,315.7773971729466 440.14285714285717,315.5327712092751 438.8571428571429,315.33174952613285 437.5714285714286,315.1753011075317 436.2857142857143,315.0643949374835 435.0,315.0 435.0,315.0 433.71428571428567,314.9827622844222 432.4285714285714,315.0120358014093 431.1428571428572,315.0868515669492 429.85714285714283,315.20624059703016 428.5714285714285,315.3692339076405 427.2857142857142,315.57486251476854 426.0000000000001,315.82215743440236 424.7142857142857,316.1101496825303 423.4285714285715,316.4378702751404 422.1428571428571,316.8043502282212 420.85714285714283,317.2086205577607 419.5714285714285,317.6497122797473 418.2857142857142,318.12665641016923 417.0,318.6384839650146 415.7142857142857,319.1842259602717 414.42857142857144,319.76291341192865 413.1428571428571,320.37357733597395 411.8571428571429,321.01524874839566 410.57142857142856,321.686958665182 409.2857142857142,322.38773810232124 407.99999999999994,323.1166180758017 406.7142857142857,323.87262960161155 405.4285714285714,324.65480369573896 404.1428571428571,325.46217137417227 402.85714285714283,326.29376365289966 401.57142857142856,327.14861154790935 400.28571428571433,328.0257460751897 399.0,328.92419825072886 397.7142857142857,329.842999090515 396.4285714285714,330.78117961053636 395.14285714285717,331.73777082678134 393.8571428571429,332.711803755238 392.57142857142856,333.7023094118947 391.2857142857143,334.70831881273955 390.0,335.72886297376095 388.7142857142858,336.7629729109469 387.42857142857144,337.80967964028594 386.14285714285717,338.86801417776604 384.8571428571429,339.9370075393756 383.57142857142856,341.01569074110273 382.2857142857143,342.103094798935

###✔️ Get Relations with Unidirectional Model

Now, let's try to get same relations with unidirectional REDL model, meaning that the model retrieves in chunk1 the left side of the relation (source), and in chunk2 the right side (target). For this, we will use the `legre_contract_doc_parties_md` model.

In [0]:
ner_model = "legner_contract_doc_parties"

re_model = "legre_contract_doc_parties_md"

model = generic_re_pipeline(ner_model, re_model)

light_model = nlp.LightPipeline(model)

result = light_model.fullAnnotate(candidates)

roberta_embeddings_legal_roberta_base download started this may take some time.
Approximate size to download 447.2 MB
[ | ][OK!]
legner_contract_doc_parties download started this may take some time.
[ | ][OK!]
legre_contract_doc_parties_md download started this may take some time.
[ | ][OK!]

In [0]:
rel_df = get_relations_df(result)

rel_df[rel_df["relation"] != "other"]

Out[50]:

,relation,entity1,entity1_begin,entity1_end,chunk1,entity2,entity2_begin,entity2_end,chunk2,confidence
0,dated_as,DOC,5,35,INTELLECTUAL PROPERTY AGREEMENT,EFFDATE,69,85,"December 31, 2018",0.9999635
1,signed_by,DOC,5,35,INTELLECTUAL PROPERTY AGREEMENT,PARTY,141,163,"Armstrong Flooring, Inc",0.9994797
2,signed_by,DOC,5,35,INTELLECTUAL PROPERTY AGREEMENT,ALIAS,192,197,Seller,0.98703974
3,signed_by,DOC,5,35,INTELLECTUAL PROPERTY AGREEMENT,PARTY,205,221,AFI Licensing LLC,0.99916875
4,signed_by,DOC,5,35,INTELLECTUAL PROPERTY AGREEMENT,ALIAS,263,271,Licensing,0.94065416
5,signed_by,DOC,5,35,INTELLECTUAL PROPERTY AGREEMENT,ALIAS,292,297,Seller,0.9914723
6,signed_by,DOC,5,35,INTELLECTUAL PROPERTY AGREEMENT,ALIAS,301,307,Arizona,0.9941164
7,signed_by,DOC,5,35,INTELLECTUAL PROPERTY AGREEMENT,PARTY,315,330,"AHF Holding, Inc",0.99897385
8,dated_as,EFFDATE,69,85,"December 31, 2018",PARTY,141,163,"Armstrong Flooring, Inc",0.7296508
9,dated_as,ALIAS,192,197,Seller,EFFDATE,69,85,"December 31, 2018",0.7790136


In [0]:
# from sparknlp_display import RelationExtractionVisualizer

re_vis = viz.RelationExtractionVisualizer()

vis = re_vis.display(result = result[0],
           relation_col = "relations",
           document_col = "document",
           exclude_relations = ["other"],
           show_relations=True,
                     return_html=True
           )

displayHTML(vis)

This INTELLECTUAL PROPERTY AGREEMENT DOC (this "Agreement"), dated as of December 31, 2018 EFFDATE (the "Effective Date") is entered into by and between Armstrong Flooring, Inc PARTY ., a Delaware corporation (" Seller ALIAS ") and AFI Licensing LLC PARTY , a Delaware limited liability company (" Licensing ALIAS " and together with Seller ALIAS , " Arizona ALIAS ") and AHF Holding, Inc PARTY . (formerly known as Tarzan HoldCo, Inc.), a Delaware corporation (" Buyer ALIAS ") and Armstrong Hardwood Flooring Company PARTY , a Tennessee corporation (the " Company ALIAS " and together with Buyer ALIAS the " Buyer Entities ALIAS ") (each of Arizona on the one hand and the Buyer Entities on the other hand, a " Party ALIAS " and collectively, the " Parties ALIAS "). <polyline fill="none" points="284.0,755.0 284.15306122448976,754.3367346938776 284.3061224489795,753.6734693877551 284.4591836734694,753.0102040816327 284.6122448979592,752.3469387755102 284.765306122449,751.6836734693877 284.9183673469388,751.0204081632652 285.07142857142867,750.3571428571431 285.2244897959184,749.6938775510206 285.3775510204082,749.0306122448982 285.53061224489795,748.3673469387755 285.6836734693877,747.7040816326529 285.8367346938775,747.0408163265306 285.98979591836735,746.3775510204082 286.1428571428571,745.7142857142859 286.295918367347,745.0510204081634 286.44897959183675,744.387755102041 286.60204081632656,743.7244897959185 286.75510204081627,743.0612244897959 286.90816326530614,742.3979591836735 287.06122448979585,741.7346938775509 287.2142857142857,741.0714285714286 287.3673469387755,740.4081632653063 287.5204081632653,739.7448979591837 287.6734693877551,739.0816326530612 287.8265306122449,738.4183673469388 287.97959183673464,737.7551020408163 288.13265306122446,737.0918367346939 288.28571428571433,736.4285714285716 288.4387755102041,735.7653061224491 288.59183673469386,735.1020408163265 288.7448979591837,734.4387755102042 288.89795918367344,733.7755102040816 289.05102040816325,733.1122448979593 289.204081632653,732.4489795918367 289.3571428571429,731.7857142857143 289.51020408163265,731.1224489795918 289.66326530612247,730.4591836734694 289.81632653061223,729.795918367347 289.96938775510205,729.1326530612246 290.12244897959187,728.469387755102 290.2755102040816,727.8061224489796 290.42857142857144,727.1428571428571 290.5816326530612,726.4795918367347 290.734693877551,725.8163265306123 290.88775510204084,725.1530612244899 291.0408163265306,724.4897959183673 291.19387755102036,723.8265306122448 291.3469387755102,723.1632653061224 291.5,722.5 291.5,722.5 291.65306122448976,721.8367346938775 291.8061224489795,721.173469387755 291.95918367346945,720.5102040816327 292.11224489795916,719.8469387755101 292.2653061224489,719.1836734693878 292.41836734693874,718.5204081632652 292.57142857142867,717.8571428571429 292.7244897959184,717.1938775510205 292.8775510204082,716.530612244898 293.03061224489795,715.8673469387754 293.18367346938766,715.204081632653 293.33673469387753,714.5408163265305 293.4897959183673,713.8775510204081 293.64285714285717,713.2142857142858 293.79591836734704,712.5510204081634 293.94897959183675,711.8877551020408 294.10204081632656,711.2244897959184 294.25510204081627,710.5612244897959 294.40816326530614,709.8979591836735 294.56122448979585,709.2346938775509 294.71428571428567,708.5714285714283 294.8673469387755,707.908163265306 295.0204081632653,707.2448979591837 295.1734693877551,706.5816326530612 295.3265306122449,705.9183673469387 295.47959183673464,705.2551020408162 295.6326530612245,704.5918367346939 295.78571428571433,703.9285714285716 295.9387755102041,703.265306122449 296.09183673469386,702.6020408163265 296.2448979591837,701.9387755102041 296.39795918367344,701.2755102040816 296.55102040816325,700.6122448979592 296.704081632653,699.9489795918366 296.8571428571429,699.2857142857143 297.01020408163265,698.6224489795918 297.1632653061224,697.9591836734694 297.3163265306123,697.295918367347 297.46938775510205,696.6326530612245 2

##✔️ Relation Extraction Model to Infer Relations Between Elements in WHEREAS Clauses

The "whereas" clause is often used in legal documents to provide background information or to set the stage for the document. It is typically used at the beginning of the document and is followed by one or more "therefore" clauses, which outline the actions or decisions that are being taken based on the information provided in the "whereas" clause.

The "whereas" clause is used to provide context and to establish a foundation for the subsequent provisions of the document. It is often used to describe the circumstances that have led to the creation of the document, or to provide other relevant information that is necessary to understand the purpose of the document.

In general, the "whereas" clause is used to provide a clear and concise explanation of the reasons behind the actions or decisions that are being taken in the document. It is an important part of many legal documents and is often used to establish a clear and logical chain of reasoning.

###📌 Firstly, we will get the `whereas` clauses
 Let's choose one WHEREAS clasue from agreement

In [0]:
candidates = [paragraphs[4], paragraphs[5], paragraphs[6], paragraphs[7], paragraphs[8], paragraphs[9]]

candidates

Out[52]: ['WHEREAS, Seller and Buyer have entered into that certain Stock Purchase Agreement, dated November 14, 2018 (the "Stock Purchase Agreement"); WHEREAS, pursuant to the Stock Purchase Agreement, Seller has agreed to sell and transfer, and Buyer has agreed to purchase and acquire, all of Seller\'s right, title and interest in and to Armstrong Wood Products, Inc., a Delaware corporation ("AWP") and its Subsidiaries, the Company and HomerWood Hardwood Flooring Company, a Delaware corporation ("HHFC," and together with the Company, the "Company Subsidiaries" and together with AWP, the "Company Entities" and each a "Company Entity") by way of a purchase by Buyer and sale by Seller of the Shares, all upon the terms and condition set forth therein;',
 "WHEREAS, Arizona owns certain Copyrights, Know-How, Patents and Trademarks which may be used in the Company Field, and in connection with the transactions contemplated by the Stock Purchase Agreement the Company desires to acquire all of Arizona's right, title and interest in and to such Intellectual Property used exclusively in the Company Field, and obtain a license from Arizona to use other such Intellectual Property on the terms and subject to the conditions set forth herein;",
 'WHEREAS, Seller is signatory to the Trademark License Agreement pursuant to which Seller obtains a license to the Arizona Licensed Trademarks;',
 'WHEREAS, the Company desires to obtain a sublicense to use the Arizona Licensed Trademarks in the Company Field;',
 'WHEREAS, Arizona has obtained consent from all counterparties to the Trademark License Agreement to grant to the Company the sublicenses to the Arizona Licensed Trademarks included in this Agreement; and',
 'WHEREAS, the Company Entities own certain Copyrights and Know-How which may be used in the Arizona Field, and in connection with the transactions contemplated by the Stock Purchase Agreement, Arizona desires to obtain a license from the Company Entities to use such Intellectual Property on the terms and subject to the conditions set forth herein.']

In [0]:
candidates = [paragraphs[9]]

candidates

Out[53]: ['WHEREAS, the Company Entities own certain Copyrights and Know-How which may be used in the Arizona Field, and in connection with the transactions contemplated by the Stock Purchase Agreement, Arizona desires to obtain a license from the Company Entities to use such Intellectual Property on the terms and subject to the conditions set forth herein.']

Previously, we used `legclf_introduction_clause_cuad` model to learn introduction of the agreement. Here we will use the `legclf_cuad_whereas_clause` model to learn WHEREAS clauses.

In [0]:
model_name = "legclf_cuad_whereas_clause"

model = generic_clf_pipeline(model_name)

sent_bert_base_cased download started this may take some time.
Approximate size to download 389.1 MB
[ | ][OK!]
legclf_cuad_whereas_clause download started this may take some time.
[ | ][OK!]

In [0]:
df = spark.createDataFrame([candidates]).toDF("text")

result = model.transform(df)

In [0]:
result.select('category.result').show()

+---------+
 result|
+---------+
[whereas]|
+---------+

📚Now, we will get relations between elements in **WHEREAS** clauses, more specifically the **SUBJECT**, the **ACTION** and the **OBJECT**. Firstly, we will extract these entities with `legner_whereas` model, after that we will extract relations between these entities with `legre_whereas` model. There are two relations possible: **has_subject** and **has_object**.

In [0]:
ner_model = "legner_whereas_md"

re_model = "legre_whereas"

model = generic_re_pipeline(ner_model, re_model)

light_model = nlp.LightPipeline(model)

result = light_model.fullAnnotate(candidates)

roberta_embeddings_legal_roberta_base download started this may take some time.
Approximate size to download 447.2 MB
[ | ][OK!]
legner_whereas_md download started this may take some time.
[ | ][OK!]
legre_whereas download started this may take some time.
[ | ][OK!]

In [0]:
## Recognized entities
pd.DataFrame([(x.result, x.metadata["entity"]) for x in result[0]["ner_chunk"]], columns=["text", "ner"])

Out[58]:

,text,ner
0,Arizona,WHEREAS_SUBJECT
1,desires to obtain,WHEREAS_ACTION
2,a license from the Company Entities,WHEREAS_OBJECT


In [0]:
rel_df = get_relations_df(result)

rel_df[rel_df["relation"] != "no_rel"]

Out[59]:

,relation,entity1,entity1_begin,entity1_end,chunk1,entity2,entity2_begin,entity2_end,chunk2,confidence
0,has_subject,WHEREAS_SUBJECT,192,198,Arizona,WHEREAS_ACTION,200,216,desires to obtain,0.6815935
1,has_subject,WHEREAS_SUBJECT,192,198,Arizona,WHEREAS_OBJECT,218,252,a license from the Company Entities,0.6681112
2,has_object,WHEREAS_ACTION,200,216,desires to obtain,WHEREAS_OBJECT,218,252,a license from the Company Entities,0.93285346


In [0]:
# from sparknlp_display import RelationExtractionVisualizer

re_vis = viz.RelationExtractionVisualizer()

vis = re_vis.display(result = result[0],
           relation_col = "relations",
           document_col = "document",
           exclude_relations = ["no_rel"],
           show_relations=True,
                     return_html=True
           )

displayHTML(vis)

WHEREAS, the Company Entities own certain Copyrights and Know-How which may be used in the Arizona Field, and in connection with the transactions contemplated by the Stock Purchase Agreement, Arizona WHEREAS_SUBJECT desires to obtain WHEREAS_ACTION a license from the Company Entities WHEREAS_OBJECT to use such Intellectual Property on the terms and subject to the conditions set forth herein. <polyline fill="none" points="308.0,255.0 306.52040816326524,253.79607986468218 305.0408163265306,252.59312871337622 303.56122448979596,251.39211553009378 302.08163265306126,250.19400929884657 300.6020408163265,248.99977900364644 299.1224489795918,247.81039362850512 297.6428571428572,246.62682215743445 296.1632653061225,245.45003357444608 294.6836734693878,244.2809968635518 293.204081632653,243.12068100876334 291.7244897959183,241.97005499409255 290.2448979591837,240.8300878035512 288.765306122449,239.70174842115102 287.2857142857143,238.58600583090382 285.80612244897964,237.4838290168213 284.32653061224494,236.39618696291515 282.84693877551024,235.32404865319725 281.36734693877554,234.26838307167935 279.88775510204084,233.23015920237316 278.4081632653061,232.21034602929052 276.9285714285714,231.20991253644314 275.44897959183675,230.22982770784284 273.96938775510205,229.2710605275013 272.48979591836735,228.33457997943037 271.0102040816327,227.42135504764173 269.53061224489795,226.53235471614718 268.05102040816325,225.6685479689585 266.5714285714286,224.83090379008752 265.0918367346939,224.0203911635458 263.6122448979592,223.23797907334531 262.1326530612245,222.4846365034977 260.6530612244898,221.7613324380148 259.1734693877551,221.0690358609083 257.6938775510204,220.40871575619002 256.2142857142858,219.78134110787175 254.73469387755105,219.18788089996517 253.25510204081633,218.6293041164821 251.77551020408166,218.10657974143427 250.29591836734693,217.6206767588335 248.81632653061226,217.1725641526915 247.3367346938776,216.76321090702004 245.8571428571429,216.3935860058309 244.37755102040816,216.06465843313583 242.89795918367346,215.77739717294665 241.4183673469388,215.53277120927507 239.9387755102041,215.33174952613282 238.4591836734694,215.1753011075317 236.9795918367347,215.06439493748354 235.5,215.0 235.5,215.0 234.02040816326527,214.98276228442228 232.54081632653057,215.0120358014092 231.06122448979593,215.08685156694915 229.58163265306126,215.20624059703016 228.1020408163265,215.36923390764048 226.62244897959178,215.57486251476848 225.14285714285717,215.82215743440238 223.66326530612247,216.11014968253025 222.18367346938774,216.43787027514048 220.70408163265301,216.8043502282212 219.22448979591837,217.20862055776075 217.74489795918365,217.64971227974735 216.26530612244898,218.1266564101692 214.78571428571425,218.63848396501453 213.30612244897964,219.18422596027168 211.82653061224488,219.7629134119287 210.34693877551024,220.37357733597398 208.86734693877548,221.01524874839563 207.38775510204079,221.68695866518203 205.90816326530606,222.38773810232126 204.4285714285714,223.1166180758017 202.94897959183675,223.87262960161155 201.46938775510205,224.654803695739 199.98979591836732,225.46217137417233 198.51020408163265,226.2937636528997 197.03061224489792,227.1486115479094 195.55102040816325,228.02574607518974 194.07142857142858,228.92419825072886 192.59183673469386,229.84299909051498 191.1122448979592,230.7811796105364 189.63265306122452,231.73777082678137 188.15306122448976,232.711803755238 186.67346938775506,233.7023094118947 185.19387755102042,234.70831881273955 183.71428571428572,235.72886297376093 182.23469387755102,236.76297291094693 180.75510204081633,237.80967964028594 179.27551020408163,238.86801417776604 177.79591836734693,239.9370075393756 176.31632653061226,241.01569074110276 174.83673469387753,242.10309479893584 173.35714285714286,243.19825072886297 171.87755102040816,244.30018954687245 170.39795918367346,245.40794226895255 168.9183673469388,246.52053991109148 167.4387755102041,247.63701348927742 165.9591836734694,248.75639401

###✔️ Get Relations with Unidirectional Model

In [0]:
ner_model = "legner_whereas_md"

re_model = "legre_whereas_md"

model = generic_re_pipeline(ner_model, re_model)

light_model = nlp.LightPipeline(model)

result = light_model.fullAnnotate(candidates)

roberta_embeddings_legal_roberta_base download started this may take some time.
Approximate size to download 447.2 MB
[ | ][OK!]
legner_whereas_md download started this may take some time.
[ | ][OK!]
legre_whereas_md download started this may take some time.
[ | ][OK!]

In [0]:
rel_df = get_relations_df(result)

rel_df[rel_df["relation"] != "other"]

Out[62]:

,relation,entity1,entity1_begin,entity1_end,chunk1,entity2,entity2_begin,entity2_end,chunk2,confidence
0,has_subject,WHEREAS_ACTION,200,216,desires to obtain,WHEREAS_SUBJECT,192,198,Arizona,0.9947596
1,has_subject,WHEREAS_OBJECT,218,252,a license from the Company Entities,WHEREAS_SUBJECT,192,198,Arizona,0.9687109
2,has_object,WHEREAS_ACTION,200,216,desires to obtain,WHEREAS_OBJECT,218,252,a license from the Company Entities,0.67824775


###📚 Visualization of Extracted Relations

In [0]:
# from sparknlp_display import RelationExtractionVisualizer

re_vis = nlp.viz.RelationExtractionVisualizer()

vis = re_vis.display(result = result[0],
           relation_col = "relations",
           document_col = "document",
           exclude_relations = ["other"],
           show_relations=True,
                     return_html=True
           )

displayHTML(vis)

WHEREAS, the Company Entities own certain Copyrights and Know-How which may be used in the Arizona Field, and in connection with the transactions contemplated by the Stock Purchase Agreement, Arizona WHEREAS_SUBJECT desires to obtain WHEREAS_ACTION a license from the Company Entities WHEREAS_OBJECT to use such Intellectual Property on the terms and subject to the conditions set forth herein. <polyline fill="none" points="163.0,255.0 164.4795918367347,253.79607986468218 165.95918367346937,252.59312871337622 167.4387755102041,251.39211553009378 168.91836734693877,250.19400929884657 170.3979591836735,248.99977900364644 171.87755102040813,247.81039362850512 173.35714285714292,246.62682215743445 174.8367346938776,245.45003357444608 176.31632653061226,244.2809968635518 177.79591836734693,243.12068100876334 179.27551020408163,241.97005499409255 180.75510204081633,240.8300878035512 182.23469387755102,239.70174842115102 183.71428571428572,238.58600583090382 185.19387755102045,237.4838290168213 186.67346938775512,236.39618696291515 188.15306122448985,235.32404865319725 189.63265306122452,234.26838307167935 191.11224489795921,233.23015920237316 192.59183673469386,232.21034602929052 194.07142857142856,231.20991253644314 195.55102040816328,230.22982770784284 197.03061224489795,229.2710605275013 198.51020408163268,228.33457997943037 199.98979591836738,227.42135504764173 201.46938775510205,226.53235471614718 202.94897959183675,225.6685479689585 204.4285714285715,224.83090379008752 205.90816326530614,224.0203911635458 207.38775510204084,223.23797907334531 208.86734693877554,222.4846365034977 210.34693877551018,221.7613324380148 211.8265306122449,221.0690358609083 213.30612244897964,220.40871575619002 214.7857142857143,219.78134110787175 216.26530612244898,219.18788089996517 217.74489795918367,218.6293041164821 219.22448979591837,218.10657974143427 220.70408163265307,217.6206767588335 222.18367346938777,217.1725641526915 223.66326530612247,216.76321090702004 225.14285714285717,216.3935860058309 226.62244897959184,216.06465843313583 228.1020408163265,215.77739717294665 229.58163265306123,215.53277120927507 231.06122448979593,215.33174952613282 232.54081632653063,215.1753011075317 234.0204081632653,215.06439493748354 235.5,215.0 235.5,215.0 236.97959183673467,214.98276228442228 238.45918367346937,215.0120358014092 239.9387755102041,215.08685156694915 241.41836734693877,215.20624059703016 242.89795918367346,215.36923390764048 244.3775510204081,215.57486251476848 245.8571428571429,215.82215743440238 247.33673469387756,216.11014968253025 248.81632653061226,216.43787027514048 250.29591836734687,216.8043502282212 251.77551020408163,217.20862055776075 253.25510204081627,217.64971227974735 254.734693877551,218.1266564101692 256.2142857142857,218.63848396501453 257.6938775510204,219.18422596027168 259.1734693877551,219.7629134119287 260.6530612244898,220.37357733597398 262.13265306122446,221.01524874839563 263.61224489795916,221.68695866518203 265.0918367346938,222.38773810232126 266.5714285714285,223.1166180758017 268.05102040816325,223.87262960161155 269.5306122448979,224.654803695739 271.01020408163265,225.46217137417233 272.4897959183673,226.2937636528997 273.969387755102,227.1486115479094 275.44897959183675,228.02574607518974 276.92857142857144,228.92419825072886 278.40816326530614,229.84299909051498 279.8877551020408,230.7811796105364 281.36734693877554,231.73777082678137 282.8469387755102,232.711803755238 284.3265306122449,233.7023094118947 285.8061224489796,234.70831881273955 287.2857142857143,235.72886297376093 288.765306122449,236.76297291094693 290.2448979591837,237.80967964028594 291.7244897959183,238.86801417776604 293.204081632653,239.9370075393756 294.6836734693877,241.01569074110276 296.1632653061224,242.10309479893584 297.64285714285717,243.19825072886297 299.1224489795918,244.30018954687245 300.6020408163265,245.40794226895255 302.0816326530612,246.52053991109148 303.5612244897959,247.63701348927742 305.0408163265306,248.75639401949869